In [1]:
# import the necessary packages
from imutils import paths
import face_recognition
import argparse
import dlib
import pickle
import cv2
import os
from keras.models import Sequential
import tensorflow
import keras
# I want to use Inception v3
import h5py
import os
import numpy as np
from keras.applications import VGG16

from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.applications.xception import preprocess_input
from keras.models import Model
from keras import backend as K
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from multiprocessing.pool import Pool

Using TensorFlow backend.


In [2]:
#!git clone https://github.com/davisking/dlib.git

In [3]:
#!git clone https://github.com/krasserm/face-recognition.git

In [4]:
# # construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--dataset", required=True,help="path to input directory of faces + images")
# ap.add_argument("-e", "--encodings", required=True,help="path to serialized db of facial encodings")
# ap.add_argument("-d", "--detection-method", type=str, default="cnn",help="face detection model to use: either `hog` or `cnn`")
# args = vars(ap.parse_args())

In [5]:
#!pip install -r requirements.txt

In [6]:
import bz2
import request
import urllib2

In [7]:
# def download_landmarks(dst_file):
#     url = 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'
#     decompressor = bz2.BZ2Decompressor()
    
#     with urllib2.urlopen(url) as src, open(dst_file, 'wb') as dst:
#         data = src.read(1024)
#         while len(data) > 0:
#             dst.write(decompressor.decompress(data))
#             data = src.read(1024)

# dst_dir = 'models'
# dst_file = os.path.join(dst_dir, 'landmarks.dat')

# if not os.path.exists(dst_file):
#     os.makedirs(dst_dir)
#     download_landmarks(dst_file)

# Coursera Implementation

In [7]:
import os
import glob
import numpy as np
import cv2
import tensorflow as tf
from fr_utils import *
import glob
from keras import backend as K

In [8]:
K.set_image_data_format('channels_first')

In [9]:
from inception_blocks_v2 import *

In [10]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [11]:
def triplet_loss(y_true, y_pred, alpha = 0.3):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,
               positive)), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, 
               negative)), axis=-1)
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
   
    return loss


In [12]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])

In [13]:
load_weights_from_FaceNet(FRmodel)

In [14]:
def preprocess_image(path):
    img = keras.preprocessing.image.load_img(path, target_size = (96,96))
    a = keras.preprocessing.image.img_to_array(img)
    #a = np.expand_dims(a, axis = 0)
    return preprocess_input(a)

In [15]:
def gs_img_encoding(img, model):
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

In [16]:
def gs_database():
    database = {}
    files= os.path.join(os.getcwd(),'Single_Image')
    for file in glob.glob(files+'/*/*'):
        identity = file.split('/')[-2]
        file=preprocess_image(file)
        database[identity] = gs_img_encoding(file, FRmodel)
    return database

In [17]:
def frame_encoding(part_image,model):
    r = 96.0 / part_image.shape[1]
    dim = (96, int(part_image.shape[0] * r)) 
    # perform the actual resizing of the image and show it
    resized = cv2.resize(part_image, dim, interpolation = cv2.INTER_AREA)
    resized=preprocess_input(resized)
    resized = np.moveaxis(resized, -1, 0)
    x_train = np.array([resized])
    embedding = model.predict_on_batch(x_train)
    return embedding

In [18]:
def who_is_it(image, database, model):
    """
    Implements face recognition for the happy house by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    encoding = frame_encoding(image, model)
    
    min_dist = 100
    identity = None
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current "emb" from the database.
        dist = np.linalg.norm(db_enc - encoding)

        print('distance for %s is %s' %(name, dist))

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name
        if dist < min_dist:
            min_dist = dist
            identity = name
    
    if min_dist > 0.90:
        return None
    else:
        return str(identity)

In [19]:
def find_identity(frame, x1, y1, x2, y2):
    """
    Determine whether the face contained within the bounding box exists in our database
    x1,y1_____________
    |                 |
    |                 |
    |_________________x2,y2
    """
    height, width, channels = frame.shape
    # The padding is necessary since the OpenCV face detector creates the bounding box around the face and not the head
    part_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]
    
    return who_is_it(part_image, database, FRmodel)

In [20]:
def process_frame(img, frame, face_cascade):
    """
    Determine whether the current frame contains the faces of people from our database
    """
    global ready_to_detect_identity
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    print("Inside process_frame function")
    # Loop through all the faces detected and determine whether or not they are in the database
    identities = []
    for (x, y, w, h) in faces:
        x1 = x-PADDING
        y1 = y-PADDING
        x2 = x+w+PADDING
        y2 = y+h+PADDING

        img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,0,0),2)

        identity = find_identity(frame, x1, y1, x2, y2)

        if identity is not None:
            identities.append(identity)
    #print("Identity",identity)
    print("Identities",identities)
    if identities != []:
        cv2.imwrite('example.png',img)
        ready_to_detect_identity = False
        #pool = Pool(processes=1) 
        # We run this as a separate process so that the camera feedback does not freeze
        #pool.apply_async(welcome_users, [identities])
    return img

In [21]:
PADDING = 50
ready_to_detect_identity = True
def webcam_face_recognizer(database):
    """
    Runs a loop that extracts images from the computer's webcam and determines whether or not
    it contains the face of a person in our database.
    If it contains a face, an audio message will be played welcoming the user.
    If not, the program will process the next frame from the webcam
    """
    global ready_to_detect_identity

    cv2.namedWindow("Face detection")
    vc = cv2.VideoCapture(0)

    face_cascade = cv2.CascadeClassifier(os.path.join(os.getcwd(),'haarcascade_frontalface_default.xml'))
    print("Inside webcam function: C1")
    while vc.isOpened():
        _, frame = vc.read()
        img = frame

        # We do not want to detect a new identity while the program is in the process of identifying another person
        if ready_to_detect_identity:
            img = process_frame(img, frame, face_cascade)   
        
        key = cv2.waitKey(100)
        cv2.imshow("preview", img)
        
        if key == 27: # exit on ESC
            break
    cv2.destroyWindow("preview")

In [22]:
database = gs_database()

In [23]:
webcam_face_recognizer(database)

Inside webcam function: C1
Inside process_frame function
distance for Tyler Nicholas is 0.71294224
distance for Brijesh Singhal is 0.89136976
distance for Nehal Jain is 0.88870996
distance for Sumanth Talasila is 0.9290208
distance for Gaurav Savlani is 1.0005174
distance for Luis Ek is 0.8006378
distance for Sudhanshu Mishra is 0.5734247
distance for Supreeth Shastry is 0.7814314
distance for Rahul Singh is 0.967438
('Identities', ['Sudhanshu Mishra'])


# Image search

In [24]:
files= os.path.join(os.getcwd(),'Single_Image')
img_1=glob.glob(files+'/*/*')[5]

In [25]:
img=cv2.imread(img_1)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# #cv2.waitkey(3)
# cv2.imshow("preview",gray)
# cv2.waitKey(20000)
# cv2.destroyAllWindows()

In [26]:
face_cascade = cv2.CascadeClassifier(os.path.join(os.getcwd(),'haarcascade_frontalface_default.xml'))
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

for (x, y, w, h) in faces:
    x1 = x-PADDING
    y1 = y-PADDING
    x2 = x+w+PADDING
    y2 = y+h+PADDING

    img = cv2.rectangle(img,(x1, y1),(x2, y2),(255,0,0),2)
    height, width, channels=img.shape
    part_image = img[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]
    cv2.imshow("preview",part_image)
    cv2.waitKey(20000)
    cv2.destroyAllWindows()

# Facial Recognition- Part 2

In [5]:
import importlib
#moduleName = input('Enter module name:')
#importlib.import_module(moduleName)

In [6]:
# 1. First create the dataset
# 2. Run encode_faces.py to build the embeddings. 
# 3. Run recognize_faces_image.py to recognize faces.

In [13]:
# USAGE
# python encode_faces.py --dataset dataset --encodings encodings.pickle

# import the necessary packages
from imutils import paths
import face_recognition
import argparse
import pickle
import cv2
import os
import glob

In [14]:
# # construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--dataset", required=True,help="path to input directory of faces + images")
# ap.add_argument("-e", "--encodings", required=True,help="path to serialized db of facial encodings")
# ap.add_argument("-d", "--detection-method", type=str, default="cnn",
#                 help="face detection model to use: either `hog` or `cnn`")
# args = vars(ap.parse_args())

In [15]:
# Creating the encoded pickle file. 
#Function to encode the names and dump encodings. 
knownEncodings = []
knownNames = []
files= os.path.join(os.getcwd(),'Single_Image')
for file in glob.glob(files+'/*/*'):
    name = file.split('/')[-2]

    # load the input image and convert it from RGB (OpenCV ordering)
    # to dlib ordering (RGB)
    image = cv2.imread(file)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # detect the (x, y)-coordinates of the bounding boxes
    # corresponding to each face in the input image
    boxes = face_recognition.face_locations(rgb,
    model="cnn")

    # compute the facial embedding for the face
    encodings = face_recognition.face_encodings(rgb, boxes)

    # loop over the encodings
    for encoding in encodings:
        # add each encoding + name to our set of known names and
        # encodings
        knownEncodings.append(encoding)
        knownNames.append(name)

# dump the facial encodings + names to disk
import pickle
print("[INFO] serializing encodings...")
data = {"encodings": knownEncodings, "names": knownNames}
f = open("Employee_Encodings.pickle", "wb")
f.write(pickle.dumps(data))
f.close()

[INFO] serializing encodings...


In [79]:
# Recognize faces in the image

In [16]:
# USAGE
# python recognize_faces_image.py --encodings encodings.pickle --image examples/example_01.png 

# import the necessary packages
import face_recognition
import argparse
import pickle
import cv2

In [ ]:
# # construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-e", "--encodings", required=True,help="path to serialized db of facial encodings")
# ap.add_argument("-i", "--image", required=True,help="path to input image")
# ap.add_argument("-d", "--detection-method", type=str, default="cnn",help="face detection model to use: either `hog` or `cnn`")
# args = vars(ap.parse_args())


In [17]:
# load the known faces and embeddings
print("[INFO] loading encodings...")
data = pickle.loads(open("Employee_Encodings.pickle", "rb").read())

[INFO] loading encodings...


In [18]:
test_file=os.path.join(os.getcwd(),"Images/Rahul Singh/Img-4.jpg")

In [19]:
# load the input image and convert it from BGR to RGB
image = cv2.imread(test_file)
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [20]:
# detect the (x, y)-coordinates of the bounding boxes corresponding
# to each face in the input image, then compute the facial embeddings
# for each face
print("[INFO] recognizing faces...")
boxes = face_recognition.face_locations(rgb,model="cnn")
encodings = face_recognition.face_encodings(rgb, boxes)

# initialize the list of names for each face detected
names = []

# loop over the facial embeddings
for encoding in encodings:
    # attempt to match each face in the input image to our known
    # encodings
    matches = face_recognition.compare_faces(data["encodings"],
        encoding)
    name = "Unknown"

    # check to see if we have found a match
    if True in matches:
        # find the indexes of all matched faces then initialize a
        # dictionary to count the total number of times each face
        # was matched
        matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        counts = {}

        # loop over the matched indexes and maintain a count for
        # each recognized face face
        for i in matchedIdxs:
            name = data["names"][i]
            counts[name] = counts.get(name, 0) + 1

        # determine the recognized face with the largest number of
        # votes (note: in the event of an unlikely tie Python will
        # select first entry in the dictionary)
        name = max(counts, key=counts.get)

    # update the list of names
    names.append(name)

[INFO] recognizing faces...


In [21]:
# loop over the recognized faces
for ((top, right, bottom, left), name) in zip(boxes, names):
    # draw the predicted face name on the image
    cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
    y = top - 15 if top - 15 > 15 else top + 15
    cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
        0.75, (0, 255, 0), 2)

# show the output image
cv2.imshow("Image", image)
cv2.waitKey(0)

27

# Using Video Streaming

In [1]:
# USAGE
# python recognize_faces_video.py --encodings encodings.pickle
# python recognize_faces_video.py --encodings encodings.pickle --output output/jurassic_park_trailer_output.avi --display 0

# import the necessary packages
from imutils.video import VideoStream
import face_recognition
import argparse
import imutils
import pickle
import time
import cv2

In [2]:
# # construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-e", "--encodings", required=True,help="path to serialized db of facial encodings")
# ap.add_argument("-o", "--output", type=str,help="path to output video")
# ap.add_argument("-y", "--display", type=int, default=1,help="whether or not to display output frame to screen")
# ap.add_argument("-d", "--detection-method", type=str, default="cnn",help="face detection model to use: either `hog` or `cnn`")
# args = vars(ap.parse_args())

In [3]:
# load the known faces and embeddings
print("[INFO] loading encodings...")
data = pickle.loads(open("Employee_Encodings.pickle", "rb").read())

[INFO] loading encodings...


In [4]:
# initialize the video stream and pointer to output video file, then
# allow the camera sensor to warm up
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
writer = None
time.sleep(2.0)

[INFO] starting video stream...


In [ ]:
# loop over frames from the video file stream
while True:
    # grab the frame from the threaded video stream
    frame = vs.read()

    # convert the input frame from BGR to RGB then resize it to have
    # a width of 750px (to speedup processing)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb = imutils.resize(frame, width=750)
    r = frame.shape[1] / float(rgb.shape[1])

    # detect the (x, y)-coordinates of the bounding boxes
    # corresponding to each face in the input frame, then compute
    # the facial embeddings for each face
    boxes = face_recognition.face_locations(rgb,model="cnn")
    encodings = face_recognition.face_encodings(rgb, boxes)
    names = []

    # loop over the facial embeddings
    for encoding in encodings:
        # attempt to match each face in the input image to our known
        # encodings
        matches = face_recognition.compare_faces(data["encodings"],
            encoding,tolerance=0.52)
        name = "Hey Random person!!"
        print(matches)
        # check to see if we have found a match
        if True in matches:
            # find the indexes of all matched faces then initialize a
            # dictionary to count the total number of times each face
            # was matched
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            print(matchedIdxs)
            counts = {}

            # loop over the matched indexes and maintain a count for
            # each recognized face face
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1
            print(counts)
            # determine the recognized face with the largest number
            # of votes (note: in the event of an unlikely tie Python
            # will select first entry in the dictionary)
            name = max(counts, key=counts.get)

        # update the list of names
        names.append(name)

    # loop over the recognized faces
    for ((top, right, bottom, left), name) in zip(boxes, names):
        # rescale the face coordinates
        top = int(top * r)
        right = int(right * r)
        bottom = int(bottom * r)
        left = int(left * r)

        # draw the predicted face name on the image
        cv2.rectangle(frame, (left, top), (right, bottom),
            (0, 255, 0), 2)
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(frame, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
            0.75, (0, 255, 0), 2)


    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

# check to see if the video writer point needs to be released
if writer is not None:
    writer.release()

[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, 

[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, False,

[False, False, False, False, False, False, False, False, False]
[False, False, True, False, False, False, False, False, False]
[2]
{'Gaurav Savlani': 1}
[False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False]
[False, False, 

# Digital Make Up

In [1]:
# import the necessary packages
from imutils.video import VideoStream
import face_recognition
import argparse
import imutils
import pickle
import time
import cv2

In [2]:
from PIL import Image, ImageDraw
import face_recognition
from IPython.display import clear_output
import webbrowser
import subprocess
import numpy

In [3]:
# initialize the video stream and pointer to output video file, then
# allow the camera sensor to warm up
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
writer = None
time.sleep(2.0)

[INFO] starting video stream...


In [4]:

# Find all facial features in all the faces in the image
i=0
while True:
    # grab the frame from the threaded video stream
    frame = vs.read()

    # convert the input frame from BGR to RGB then resize it to have
    # a width of 750px (to speedup processing)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb = imutils.resize(frame, width=750)
    r = frame.shape[1] / float(rgb.shape[1])

    face_landmarks_list = face_recognition.face_landmarks(rgb)
    for face_landmarks in face_landmarks_list:
        pil_image = Image.fromarray(rgb)
        d = ImageDraw.Draw(pil_image, 'RGBA')

        # Make the eyebrows into a nightmare
        d.polygon(face_landmarks['left_eyebrow'], fill=(68, 54, 39, 128))
        d.polygon(face_landmarks['right_eyebrow'], fill=(68, 54, 39, 128))
        d.line(face_landmarks['left_eyebrow'], fill=(68, 54, 39, 150), width=5)
        d.line(face_landmarks['right_eyebrow'], fill=(68, 54, 39, 150), width=5)

        # Gloss the lips
        d.polygon(face_landmarks['top_lip'], fill=(150, 0, 0, 128))
        d.polygon(face_landmarks['bottom_lip'], fill=(150, 0, 0, 128))
        d.line(face_landmarks['top_lip'], fill=(150, 0, 0, 64), width=8)
        d.line(face_landmarks['bottom_lip'], fill=(150, 0, 0, 64), width=8)

        # Sparkle the eyes
        d.polygon(face_landmarks['left_eye'], fill=(255, 255, 255, 30))
        d.polygon(face_landmarks['right_eye'], fill=(255, 255, 255, 30))

        # Apply some eyeliner
        d.line(face_landmarks['left_eye'] + [face_landmarks['left_eye'][0]], fill=(0, 0, 0, 110), width=6)
        d.line(face_landmarks['right_eye'] + [face_landmarks['right_eye'][0]], fill=(0, 0, 0, 110), width=6)
        open_cv_image = numpy.array(pil_image) 
        # Convert RGB to BGR 
        open_cv_image = open_cv_image[:, :, ::-1].copy() 
        # check to see if we are supposed to display the output frame to
        # the screen
        if 1 > 0:
            print('Displaying frame:',i)
            cv2.imshow("Frame", open_cv_image)
            i+=1
            key = cv2.waitKey(1) & 0xFF
            # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

('Displaying frame:', 0)
('Displaying frame:', 1)
('Displaying frame:', 2)
('Displaying frame:', 3)
('Displaying frame:', 4)
('Displaying frame:', 5)
('Displaying frame:', 6)
('Displaying frame:', 7)
('Displaying frame:', 8)
('Displaying frame:', 9)
('Displaying frame:', 10)
('Displaying frame:', 11)
('Displaying frame:', 12)
('Displaying frame:', 13)
('Displaying frame:', 14)
('Displaying frame:', 15)
('Displaying frame:', 16)
('Displaying frame:', 17)
('Displaying frame:', 18)
('Displaying frame:', 19)
('Displaying frame:', 20)
('Displaying frame:', 21)
('Displaying frame:', 22)
('Displaying frame:', 23)
('Displaying frame:', 24)
('Displaying frame:', 25)
('Displaying frame:', 26)
('Displaying frame:', 27)
('Displaying frame:', 28)
('Displaying frame:', 29)
('Displaying frame:', 30)
('Displaying frame:', 31)
('Displaying frame:', 32)
('Displaying frame:', 33)
('Displaying frame:', 34)
('Displaying frame:', 35)
('Displaying frame:', 36)
('Displaying frame:', 37)
('Displaying frame:', 

KeyboardInterrupt: 